In [ ]:
#pip3 install pycuda
#pip3 install matplotlib

In [ ]:
!mkdir ./figs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit

cuda_code =  SourceModule("""
     #include <stdio.h>

   __device__ inline int pbc(int x, int L);
   
   __global__ void diffusion(double *arr_in, 
                             double *arr_out,
                             double D,
                             int nr, 
                             int nc)
    {
      
      int i = threadIdx.x+ blockIdx.x* blockDim.x;
      int j = threadIdx.y+ blockIdx.y* blockDim.y;

      int n = pbc(i-1,nr);
      int s = pbc(i+1,nr);
      int w = pbc(j-1,nc);
      int e = pbc(j+1,nc);

      arr_out[i*nc+j] =  arr_in[i*nc+j] +
                         D*(arr_in[n*nc+j] + arr_in[s*nc+j] +
                            arr_in[i*nc+w] + arr_in[i*nc+e] -
                            4.0*arr_in[i*nc+j]);
    }

    __device__ inline int pbc(int x, int L)
    {
        return  x - (int)floor((double)x/L)*L;
    }

""")


def plot_heat_map(matrix, mx, mn, save=False, saveid=0):
  """
      plot_heat_map
      Plots a heatmap or save it to file
      
      Input:    matrix = 2D array to reppresent
                mx = max value to scale
                mn = min value to scale
                save = Boolean to indicate if is saved or reppresented
                id = If saved the file is going to be saved with id + name
  """
  # Show the array
  fig = plt.figure()
  hm=plt.imshow(matrix, 
                cmap='gray', 
                interpolation ='none', 
                aspect = 'auto',  
                vmin=mn, vmax=mx)
  plt.colorbar(hm)
  plt.axes().set_aspect(1.0)

  if save==False:
    plt.ion()
    plt.show()
  else:
    backend_= mpl.get_backend() 
    mpl.use("Agg")  # Prevent showing the plots in Jupyter
    plt.savefig("./figs/" + str(saveid) + "_diffusion.png")   
    plt.close()
    mpl.use(backend_) # Reset backend



def initilise_array(arr, h, w, r, c0):
  """
      initialise_array
      Generates a drop in the middle of the array

      Input:    arr = w x h array to initialise
                h = height, number of rows
                w = width, number of columns
                r = radius of the drop
                c0 = Initial concentration

      Ouput     arr = Output array with the drop
  """

  wh2 = w//2
  hh2 = h//2
  for i in range(0,h):
    for j in range(0,w):
      x=i-hh2
      y=j-wh2
      if(np.sqrt(x**2 + y**2)<r):
        arr[i,j] = c0



def diffusion(w, h, D, maxtime):
  """
      diffusion
      This function calls to the CUDA module to calculate the laplacian and the
      ensuing diffusion

      Input:       h = height, number of rows
                   w = width, number of columns
                   D = Diffusion constant (expressed in units of latice sites)
                   maxtime = Number of iterations
  """
  # We need to explicitely transform to the specific data types because CUDA variables 
  # are type-specific
  w=np.int32(w)
  h=np.int32(h)
  D=np.float64(D)
  
  # Get the function from the CUDA module
  laplacian = cuda_code.get_function("diffusion") 

  # Allocate data in the computer + initilisation
  arr = np.zeros((h,w), dtype=np.float64)
  c0 = 10.0
  r = 60
  initilise_array(arr, h, w, r, c0)
  plot_heat_map(arr, c0, 0.0, False)
  
  # Allocate memory for the array in the GPU
  arr_gpu     = cuda.mem_alloc(arr.nbytes) 
  arr_upd_gpu = cuda.mem_alloc(arr.nbytes) 
  cuda.memcpy_htod(arr_gpu, arr)

  # Iterate the function on the GPU 
  # We first define the minimal unit ie, the block size. A grid is then a 
  # a collection of blocks, so we attribute a number of blocks that divisible with 
  # with the block size considering the size of the array. 
  bw=32   
  bh=32
  gw=int(np.floor(w/bw))
  gh=int(np.floor(h/bh))
  save_freq=20
  for t in range(0, maxtime):
    # Send data from computer to GPU 
    cuda.memcpy_htod(arr_gpu, arr)

    # Execute the function on the GPU 
    laplacian(arr_gpu, arr_upd_gpu, D, h, w, block=(bw,bh,1), grid=(gw,gh))
    
    # Retrieve the data from the GPU to the computer
    cuda.memcpy_dtoh(arr, arr_upd_gpu)

    # Save
    if t%save_freq==0:
      plot_heat_map(arr, c0, 0.0, True, round(t/save_freq))

   
  plot_heat_map(arr, c0, 0.0, False)
  return 0



# if __name__ == "__main__":
D = 0.2
w = 512
h = 512
maxtime = 50000
diffusion(w, h, D, maxtime)


In [ ]:
!zip -r -q figs.zip ./figs
!rm -r ./figs